**Importar datos con API de Kaggle**

**Extracción de Datos**

In [15]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine

In [7]:
# Cargar los datasets
orders = pd.read_csv("raw_data/olist_orders_dataset.csv")
order_items = pd.read_csv("raw_data/olist_order_items_dataset.csv")
customers = pd.read_csv("raw_data/olist_customers_dataset.csv")
sellers = pd.read_csv("raw_data/olist_sellers_dataset.csv")
products = pd.read_csv("raw_data/olist_products_dataset.csv")
payments = pd.read_csv("raw_data/olist_order_payments_dataset.csv")




# Ver las primeras filas de cada dataset
print("Orders:\n", orders.head(), "\n")
print("Order Items:\n", order_items.head(), "\n")
print("Customers:\n", customers.head(), "\n")
print("Sellers:\n", sellers.head(), "\n")
print("Products:\n", products.head(), "\n")
print("Payments:\n", payments.head(), "\n")


Orders:
                            order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00         

**Revisar datos**

In [8]:
datasets = {
    "orders": orders,
    "order_items": order_items,
    "customers": customers,
    "sellers": sellers,
    "products": products,
    "payments": payments,
}

for name, df in datasets.items():
    print(f"{name.upper()} - Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
    print(df.describe(), "\n")  


ORDERS - Filas: 99441, Columnas: 8
                                order_id                       customer_id  \
count                              99441                             99441   
unique                             99441                             99441   
top     66dea50a8b16d9b4dee7af250b4be1a5  edb027a75a1449115f6b43211ae02a24   
freq                                   1                                 1   

       order_status order_purchase_timestamp    order_approved_at  \
count         99441                    99441                99281   
unique            8                    98875                90733   
top       delivered      2018-08-02 12:05:26  2018-02-27 04:31:10   
freq          96478                        3                    9   

       order_delivered_carrier_date order_delivered_customer_date  \
count                         97658                         96476   
unique                        81018                         95664   
top             2018-

**Transformación de datos**

**Varifiación y Limpieza de datos**

In [9]:
for name, df in datasets.items():
    print(f"{name.upper()} - Valores nulos:")
    print(df.isnull().sum(), "\n")
    
    print(f"{name.upper()} - Duplicados: {df.duplicated().sum()} \n")
    print("=" * 50)


ORDERS - Valores nulos:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64 

ORDERS - Duplicados: 0 

ORDER_ITEMS - Valores nulos:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64 

ORDER_ITEMS - Duplicados: 0 

CUSTOMERS - Valores nulos:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64 

CUSTOMERS - Duplicados: 0 

SELLERS - Valores nulos:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64 

SELLERS - D

**Eliminar datos faltantes y tipos de datos**

In [10]:
#Limpieza de de datos faltantes
for name,df in datasets.items():
    df.dropna(inplace=True)
    print(f"{name.upper()} - Valores nulos:")
    print(df.isnull().sum(), "\n")
    
    df.drop_duplicates(inplace=True)
    print(f"{name.upper()} - Duplicados: {df.duplicated().sum()} \n")
    print("=" * 50)

ORDERS - Valores nulos:
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64 

ORDERS - Duplicados: 0 

ORDER_ITEMS - Valores nulos:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64 

ORDER_ITEMS - Duplicados: 0 

CUSTOMERS - Valores nulos:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64 

CUSTOMERS - Duplicados: 0 

SELLERS - Valores nulos:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64 

SELLERS - Duplicados: 0 

PRODUCTS 

In [11]:
#Asegurar tipo de datos fecha
#Revisar tipo de datos 
for name, df in datasets.items():
    print(f"{name.upper()} - Tipos de datos:")
    print(df.dtypes, "\n")


ORDERS - Tipos de datos:
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object 

ORDER_ITEMS - Tipos de datos:
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object 

CUSTOMERS - Tipos de datos:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object 

SELLERS - Tipos de datos:
seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object

In [12]:
# Convertir columnas de fecha a datetime
orders[[
    "order_purchase_timestamp", 
    "order_approved_at", 
    "order_delivered_carrier_date", 
    "order_delivered_customer_date", 
    "order_estimated_delivery_date"
]] = orders[[
    "order_purchase_timestamp", 
    "order_approved_at", 
    "order_delivered_carrier_date", 
    "order_delivered_customer_date", 
    "order_estimated_delivery_date"
]].apply(pd.to_datetime)

order_items["shipping_limit_date"] = pd.to_datetime(order_items["shipping_limit_date"])

# Convertir códigos postales a string
customers["customer_zip_code_prefix"] = customers["customer_zip_code_prefix"].astype(str)
sellers["seller_zip_code_prefix"] = sellers["seller_zip_code_prefix"].astype(str)

# Convertir columnas numéricas con valores nulos a enteros
products[[
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
]] = products[[
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
]].astype("Int64")  # Permite valores nulos en enteros

# Verificar cambios
datasets = {
    "ORDERS": orders,
    "ORDER_ITEMS": order_items,
    "CUSTOMERS": customers,
    "SELLERS": sellers,
    "PRODUCTS": products,
    "PAYMENTS": payments,
}

for name, df in datasets.items():
    print(f"{name} - Tipos de datos:\n{df.dtypes}\n")


ORDERS - Tipos de datos:
order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

ORDER_ITEMS - Tipos de datos:
order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

CUSTOMERS - Tipos de datos:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix    object
customer_city               object
customer_state              object
dtype: object

SELLERS - Tipos de datos:
seller_id            

In [13]:
# Crear la carpeta si no existe
os.makedirs("processed_data", exist_ok=True)

# Guardar los DataFrames transformados
customers.to_csv("processed_data/customers.csv", index=False)
sellers.to_csv("processed_data/sellers.csv", index=False)
products.to_csv("processed_data/products.csv", index=False)
orders.to_csv("processed_data/orders.csv", index=False)
order_items.to_csv("processed_data/order_items.csv", index=False)
payments.to_csv("processed_data/payments.csv", index=False)

print("Archivos guardados en la carpeta 'processed_data'.")

Archivos guardados en la carpeta 'processed_data'.


**Carga de datos**

In [ ]:
# Parámetros de conexión
DB_NAME = "Ecommerce_DB"
DB_USER = "soporte"
DB_HOST = "localhost"
DB_PASSWORD = "contraseña"  # Reemplaza con la contraseña real

# Conexión a PostgreSQL
try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        host=DB_HOST,
        password=DB_PASSWORD
    )
    print("Conexión exitosa a PostgreSQL")
    conn.close()
except Exception as e:
    print("Error en la conexión:", e)


Conexión exitosa a PostgreSQL


In [ ]:
# Configurar la conexión a PostgreSQL
db_user = "soporte"
db_password = "contraseña"  # Cambia esto por tu contraseña real
db_host = "localhost"
db_port = "5432"
db_name = "Ecommerce_DB"

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Cargar los archivos transformados
customers = pd.read_csv("processed_data/customers.csv")
sellers = pd.read_csv("processed_data/sellers.csv")
products = pd.read_csv("processed_data/products.csv")
orders = pd.read_csv("processed_data/orders.csv", parse_dates=[
    "order_purchase_timestamp", "order_approved_at",
    "order_delivered_carrier_date", "order_delivered_customer_date",
    "order_estimated_delivery_date"
])
order_items = pd.read_csv("processed_data/order_items.csv", parse_dates=["shipping_limit_date"])
payments = pd.read_csv("processed_data/payments.csv")

# Insertar los datos en PostgreSQL
customers.to_sql("customers", engine, if_exists="append", index=False)
sellers.to_sql("sellers", engine, if_exists="append", index=False)
products.to_sql("products", engine, if_exists="append", index=False)
orders.to_sql("orders", engine, if_exists="append", index=False)
order_items.to_sql("order_items", engine, if_exists="append", index=False)
payments.to_sql("payments", engine, if_exists="append", index=False)

engine.dispose()
print("Carga de datos completada con éxito.")


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "payments" violates foreign key constraint "payments_order_id_fkey"
DETAIL:  Key (order_id)=(5d9c5817e278892b7498d90bfa28ade8) is not present in table "orders".

[SQL: INSERT INTO payments (order_id, payment_sequential, payment_type, payment_installments, payment_value) VALUES (%(order_id__0)s, %(payment_sequential__0)s, %(payment_type__0)s, %(payment_installments__0)s, %(payment_value__0)s), (%(order_id__1)s, %(pa ... 127208 characters truncated ... ent_sequential__999)s, %(payment_type__999)s, %(payment_installments__999)s, %(payment_value__999)s)]
[parameters: {'order_id__0': 'b81ef226f3fe1789b1e8b2acac839d17', 'payment_value__0': 99.33, 'payment_type__0': 'credit_card', 'payment_installments__0': 8, 'payment_sequential__0': 1, 'order_id__1': 'a9810da82917af2d9aefd1278f1dcfa0', 'payment_value__1': 24.39, 'payment_type__1': 'credit_card', 'payment_installments__1': 1, 'payment_sequential__1': 1, 'order_id